# Objectives
- Understand 3 main groups of data:
    - Customer (Broker) Demographic data
    - Advertising Campaign data
    - Success Marker data

# Week 2 : Data Cleaning
Assigned DataSets:"Goal stats - web traffic", "General stats - web traffic"

0. Load dataset
1. Change column headings to names that are easier to reference
2. Explore the data.
    1. Create a new DataFrame.
    2. Sense-check the DataFrame.
    3. Determine if there are any missing values in the DataFrame.
    4. Create a summary of the descriptive statistics.
3. Remove redundant columns
4. Save a copy of the clean DataFrame as a CSV file. Import the file to sense-check.

## 0. Load file & create dataframes (GenStats)


In [1]:
# Imports
import numpy as np
import pandas as pd

In [2]:
GenStats_raw = pd.read_excel("Change 2022_GA writeback_091122.xlsx", sheet_name="General stats - web traffic")

GenStats_raw.head()

,Date,Audience,Creative - Family,Creative - Version,Platform,Ad Format,Campaign Traffic?,Total Sessions,Total Bounces,Total Duration,Days away from max date,Latest report?
0,2022-08-16,NaN,CloserTwins,Cutdown1A,Domain Display,Video,Campaign,1,0,73.0,76,0
1,2022-06-16,3,CloseFaster,NoDTI,Facebook,Single image,Campaign,1,0,0.0,137,0
2,2022-08-29,1,CloseFaster,MoreAll,User ID Display,NaN,Campaign,2,0,0.0,63,0
3,2022-06-09,4,UnfairAdvantage,1page,LinkedIn,Single image,Campaign,2,0,50.0,144,0
4,2022-08-03,1,UnfairAdvantage,1099,Domain Display,Single image,Campaign,1,0,0.0,89,0


In [3]:
#Create new dataframe for cleaned data
GenStats = GenStats_raw.copy()

## 1. Rename Columns

In [4]:
# Rename the column headers.
GenStats = GenStats.rename(
    columns={
        "City, Country": "Location",
        "Creative - Family": "Creative_Family",
        "Creative - Version": "Creative_Version",
        "Ad Format": "Ad_Format",
        "Campaign Traffic?": "Campaign_Traffic",
        "Total Sessions": "Total_Sessions",
        "Total Bounces": "Total_Bounces",
        "Total Duration": "Total_Duration",
        "Days away from max date": "Days_Max_Date",
        "Latest report?": "Latest_Report"})



## 1. Check for missing values
- Evaluate what to do with entries with missing values

In [5]:
GenStats_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   Date                     13194 non-null  datetime64[ns]
 1   Audience                 12430 non-null  object        
 2   Creative - Family        13194 non-null  object        
 3   Creative - Version       13194 non-null  object        
 4   Platform                 12821 non-null  object        
 5   Ad Format                7956 non-null   object        
 6   Campaign Traffic?        13194 non-null  object        
 7   Total Sessions           13194 non-null  int64         
 8   Total Bounces            13194 non-null  int64         
 9   Total Duration           13194 non-null  float64       
 10  Days away from max date  13194 non-null  int64         
 11  Latest report?           13194 non-null  int64         
dtypes: datetime64[ns](1), float64(1)

## Basic Overview:

Out of __13194 entries__

There are missing data for:
- Audience : **764** missing values [**5.8%** missing]
    - Unable to determine which demographic group entry falls under
    - Will not be useful in determining effectiveness of ad campaigns among each demographic group<br>
> 402 (Campaign Traffic), 362 (General Traffic)
> ⇒ Blanks to be replaced with with NA value for 'Audience' column, however, data will likely be excluded in later analysis
> 

- Platform :  **373** missing values [**2.8%** missing]
    - However, Ad campaign family & Version information still available <br>
    - Entries can still be used ⇒ no need to remove whole row <br>
>⇒ **Replace NA entries with "NA"** for 'Platform' column

 <br>  
- Ad Format :  **5238** missing values [**39.7%** missing]
    - Significant number of entries with missing data for 'Ad Format'
    - Significantly reduce sample size if all the data removed
>⇒ **Replace NA entries with "NA"** for 'Platform' column

In [6]:
# Create function to check unique values in 
def col_list(df, para):
    print("List of values in <", para ,"> : \n",df[para].unique(),"\n")
    return

### 1.1. Demographic Information
Columns: 'Audience' <br>

 1   Audience                 12430 non-null  object       
 
 Expected Values: <br>
Audience 1	:	Registered Loan Officers from Registered Brokerage, active (last 120 days). <br>
Audience 2	:	Registered Loan Officers from Registered Brokerage, inactive (last 120 days). <br>
Audience 3	:	Registered Loan Officers from Registered Brokerage, never registered a loan. <br>
Audience 4	:	Non-Registered Loan Officers from Registered Brokerages. <br>
Audience 5	:	Retargeted audience. Non-Registered Loan Officers who visited website (last 7 days). <br>
Audience 6	:	General Targeting. Brokers not registered and not from registered brokerages.

In [7]:
col_list(GenStats, 'Audience')

List of values in < Audience > : 
 [nan '3' '1' '4' '5' 'General Targetting' '2'
 '1to4https://changewholesale.com/anti-inflation-special/?utm_campaign=FY22_anti_inflation'
 '1to4'
 'fourhttps://changewholesale.com/broker-approval/?utm_campaign=FY23_broker_campaign'
 'fivehttps://changewholesale.com/broker-approval/?utm_campaign=FY23_broker_campaign'
 'fivehttps://changewholesale.com/closer-twins/' '44652' 'test2'
 'fivedisparate' 'test3'
 'fourhttps://changewholesale.com/?utm_campaign=FY23_broker_campaign'
 'test20th' 'five/broker-approval/' 'test6' 'one/' 'test5'] 



### Cleaning Required:  
1) Remove unwanted URLs attached to numbers (1-6)  <br>
2) Rename "1to4" to "4"
> Based on preliminary check, Audience 4 is the most common

3) Rename "General Targetting" to "6" <br>
4) Remove all none 1-6 values <br>

In [8]:
# Replace numbers with unwanted tail
GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*one.*$)', '1')
GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*four.*$)', '4')
GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*five.*$)', '5')

# Replace '1to4' with '4'
GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*1to4.*$)', '4')

# Rename 'General Targetting' to '6'
GenStats['Audience'] = GenStats.Audience.str.replace('General Targetting', '6')

# Replace all non-'1-6' values
allowed_vals = ['1','2','3','4','5','6']
GenStats.loc[~GenStats['Audience'].isin(allowed_vals), 'Audience'] = 'NA'

# Check remaining values 
col_list(GenStats, 'Audience')

List of values in < Audience > : 
 ['NA' '3' '1' '4' '5' '6' '2'] 



C:\Users\athir\AppData\Local\Temp/ipykernel_21972/3115880027.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*one.*$)', '1')
C:\Users\athir\AppData\Local\Temp/ipykernel_21972/3115880027.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*four.*$)', '4')
C:\Users\athir\AppData\Local\Temp/ipykernel_21972/3115880027.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*five.*$)', '5')
C:\Users\athir\AppData\Local\Temp/ipykernel_21972/3115880027.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  GenStats['Audience'] = GenStats.Audience.str.replace(r'(^.*1to4.*$)', '4')


In [9]:
GenStats.groupby('Audience')\
        .agg({'Total_Sessions':sum,'Total_Bounces':sum,'Total_Duration':sum})\
        .sort_values('Total_Sessions', ascending=False)

,Total_Sessions,Total_Bounces,Total_Duration
Audience,,,
NA,446497,52532,1.345429e+08
4,23548,13,2.509180e+05
6,21026,23,2.210070e+06
5,16385,1559,4.282180e+05
1,8349,3,3.721500e+04
3,7484,0,1.969400e+04
2,4598,0,1.108600e+04


In [10]:
GenStats.groupby('Audience')['Audience'].count()

Audience
1     2041
2     1174
3     1487
4     3708
5     2851
6     1162
NA     771
Name: Audience, dtype: int64

### Re-evaluating the need to remove missing data under 'Audience'

Only 5.8% of entries have 'NA' audience type.
However, this group also accounts for 84.6% of sessions and 97.8% of total duration.

> I have opted to keep these entries until further discussion but they can still be easily removed afterwards by deleting all 'NA' entries

### 1.2. Advertising Campaign
Columns:  <br>
 2   Creative - Family        13194 non-null  object         <br>
 3   Creative - Version       13194 non-null  object         <br>
 4   Platform                 12821 non-null  object         <br>
 5   Ad Format                7956 non-null   object         <br>
 6   Campaign Traffic?        13194 non-null  object         <br>
 
 Expected Values: <br>

In [11]:
col_list(GenStats, 'Creative_Family')
col_list(GenStats, 'Creative_Version')
col_list(GenStats, 'Platform')
col_list(GenStats, 'Ad_Format')
col_list(GenStats, 'Campaign_Traffic')

List of values in < Creative_Family > : 
 ['CloserTwins' 'CloseFaster' 'UnfairAdvantage' 'SEM Ads' 'Trade Media Ads'
 '(not set)' datetime.datetime(2022, 6, 9, 0, 0) 'domain' 'August'
 'August/' 'CompetitiveOpportunity' '08-29-2022' 'newsletter' 'NovDec'
 'DybffeGjvaf' 'One-Off' datetime.datetime(2022, 1, 3, 0, 0) 'crm'
 'ComingSoon' 'All3' 227112117 '08-25-2022' '08-24-2022' 'SnapdocsLive'
 'eml' '08-30-2022' datetime.datetime(2022, 8, 9, 0, 0) '10-28-2022'
 'ebgf' 'December' 44801 206306768 'nmls' 'Baf-Baa' 'DbzvatFbba' 'unfair'
 'Bhthfg' 'FabcebdfYvif' 219526440 'afjfyfggfe' '08-28-2022'] 

List of values in < Creative_Version > : 
 ['Cutdown1A' 'NoDTI' 'MoreAll' '1page' 1099 'MoreLoansAll' 'Faceoff' 'All'
 "We Are America's CDFI" 'EarlyBird' 'OnePage' 'CloseMore' 'ROS5'
 'Competitors' 'Namaste' 'interactive' 'OTT_15' 'ROS1' 'Change Wholesale'
 'FasterAll' '3steps' 'Paperwork' '300x250' 'Faster' 'Cutdown1B'
 'Animated' '(not set)' 'Faceoff1' 'FasterReg' 'RTB'
 datetime.datetime(2022

### Cleaning Required:  
1) No cleaning required for 'Campaign Traffic?'  <br>
2) Rename missing values in 'Platform', 'Ad Format' with "NA" <br>
<br>
3) Many values in 'Creative - Family' and 'Creative - Version' <br>
    - will need to evaluate which are actual values and which are erroneous <br>
    - Maybe cross-refer to other datasets

### 1.2.1 Platform & Ad_Format

In [12]:
# Replace missing values
GenStats = GenStats.fillna('NA')

col_list(GenStats, 'Platform')
col_list(GenStats, 'Ad_Format')

List of values in < Platform > : 
 ['Domain Display' 'Facebook' 'User ID Display' 'LinkedIn' 'Google SEM'
 'Trade Media' 'OTT' 'NA'] 

List of values in < Ad_Format > : 
 ['Video' 'Single image' 'NA' 'Carousel' 'CPC' 'Housingwire'
 'National_mortgage_news' 'Animated' 'Inside_mortgage_finance_newsletter'
 'Chrisman' 'Scotsman' 'Nmn' 'Nmn_partner_insight_1'
 'Nmn_partner_insight_2'] 



### 1.2.2 Creative_Family & Creative_Version
__For 'Creative - Family':__ <br>
After evaluating the values in the 'Creative_Family' column, we have found that most of the illogical values are for 'General Traffic' records. Since 'General Traffic' would not be associated with any campaign, we have opted only to keep the "Creative_Family" values that are associated with the campaign traffic with the same list as Goal-Stats

> __Accepted values:__
SEM Ads, 
UnfairAdvantage, 
CloserTwins, 
Trade Media Ads, 
CloseFaster, 
domain, 
August, 
CompetitiveOpportunity, 
newsletter, 
crm
 <br>

__For 'Creative - Version':__<br>
We have opted to leave the values as is, since we cannot be sure which version names are correct
> Column left untouched so that the data can be used if needed <br>
> However, we currently have no plans to use this data column

__For both columns:__<br>
> All blanks & '(not set)' values to be renamed to 'NA' for consistency

In [13]:
# Original values for Creative_Family in GenStats
col_list(GenStats, 'Creative_Family')

List of values in < Creative_Family > : 
 ['CloserTwins' 'CloseFaster' 'UnfairAdvantage' 'SEM Ads' 'Trade Media Ads'
 '(not set)' datetime.datetime(2022, 6, 9, 0, 0) 'domain' 'August'
 'August/' 'CompetitiveOpportunity' '08-29-2022' 'newsletter' 'NovDec'
 'DybffeGjvaf' 'One-Off' datetime.datetime(2022, 1, 3, 0, 0) 'crm'
 'ComingSoon' 'All3' 227112117 '08-25-2022' '08-24-2022' 'SnapdocsLive'
 'eml' '08-30-2022' datetime.datetime(2022, 8, 9, 0, 0) '10-28-2022'
 'ebgf' 'December' 44801 206306768 'nmls' 'Baf-Baa' 'DbzvatFbba' 'unfair'
 'Bhthfg' 'FabcebdfYvif' 219526440 'afjfyfggfe' '08-28-2022'] 



In [14]:
CreFam_allowed = ['SEM Ads', 'UnfairAdvantage', 'CloserTwins', \
                  'Trade Media Ads', 'CloseFaster', 'domain', 'August', \
                  'CompetitiveOpportunity', 'newsletter', 'crm']

In [15]:
# 'NA' all values that are not in the allowed list
GenStats.loc[~GenStats['Creative_Family'].isin(CreFam_allowed), \
             'Creative_Family'] = 'NA'

col_list(GenStats, 'Creative_Family')

List of values in < Creative_Family > : 
 ['CloserTwins' 'CloseFaster' 'UnfairAdvantage' 'SEM Ads' 'Trade Media Ads'
 'NA' 'domain' 'August' 'CompetitiveOpportunity' 'newsletter' 'crm'] 



In [16]:
# Rename '(not set)' to 'NA'
GenStats['Creative_Version'] = GenStats.Creative_Version.replace('(not set)', 'NA')

col_list(GenStats, 'Creative_Version')

List of values in < Creative_Version > : 
 ['Cutdown1A' 'NoDTI' 'MoreAll' '1page' 1099 'MoreLoansAll' 'Faceoff' 'All'
 "We Are America's CDFI" 'EarlyBird' 'OnePage' 'CloseMore' 'ROS5'
 'Competitors' 'Namaste' 'interactive' 'OTT_15' 'ROS1' 'Change Wholesale'
 'FasterAll' '3steps' 'Paperwork' '300x250' 'Faster' 'Cutdown1B'
 'Animated' 'NA' 'Faceoff1' 'FasterReg' 'RTB'
 datetime.datetime(2022, 6, 9, 0, 0) 'Close More. Close Faster.'
 'MoreNoReg' 'ad1' 'Faceoff2' 'MoreLoans' 'August' 'ShapeUp' 'OTT_30QR'
 'August/' 'Cancelingyourlock' 'Rate Lock' '08-29-2022'
 'Theycancelweclose' 'OTT_30' 'NovDec' '728x90' 'EGC' 'One-Off'
 datetime.datetime(2022, 1, 3, 0, 0) 'ComingSoon' 'All3' 227112117
 '08-25-2022' '08-24-2022' 'OTT_15QR' 'ROS2' 'SnapdocsLive' '08-30-2022'
 datetime.datetime(2022, 8, 9, 0, 0) '10-28-2022' 'wallpaper' 'ybdx'
 'December' 44801 206306768 'Baf-Baa' 'ROS' 'DbzvatFbba' 'ROP' 'adv'
 'Bhthfg' 'FabcebdfYvif' 219526440 '633k583' '08-28-2022'] 



### 1.3. Success Markers
Columns:  <br>        
 7   Total Sessions           13194 non-null  int64         <br>
 8   Total Bounces            13194 non-null  int64         <br>
 9   Total Duration           13194 non-null  float64       <br>
 
 Expected Values: <br>
 
 > Since all values in these columns are numbers, we can get a number sensing

In [17]:
GenStats.describe()

,Total_Sessions,Total_Bounces,Total_Duration,Days_Max_Date,Latest_Report
count,13194.000000,13194.000000,1.319400e+04,13194.000000,13194.000000
mean,40.009626,4.102622,1.042141e+04,97.686600,0.063741
std,317.764981,39.480841,1.043190e+05,56.556894,0.244300
min,1.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,1.000000,0.000000,0.000000e+00,49.000000,0.000000
50%,2.000000,0.000000,0.000000e+00,94.000000,0.000000
75%,6.000000,0.000000,0.000000e+00,146.000000,0.000000
max,6505.000000,608.000000,1.737539e+06,213.000000,1.000000


__Notable values__ <br>
Total_Duration = 0 for at least the bottom 75 percentiles <br>
➜ Even though Total_bounce = 0 for most of these entries (10182 out of 10278 [99.1%])
> Might need to revisit whether values for duration = 0 is usable or erroneous

### Cleaning Required:  
1) All measures already in numerical data type, no change required  <br>
2) No missing values 

## 3. Remove Redundant Columns

Unused: 'Days_Max_Date','Latest_Report'

In [18]:
GenStats = GenStats.drop(columns=['Days_Max_Date','Latest_Report'])

## 4. Save cleaned dataframe as csv

In [19]:
GenStats.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13194 entries, 0 to 13193
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Date              13194 non-null  datetime64[ns]
 1   Audience          13194 non-null  object        
 2   Creative_Family   13194 non-null  object        
 3   Creative_Version  13194 non-null  object        
 4   Platform          13194 non-null  object        
 5   Ad_Format         13194 non-null  object        
 6   Campaign_Traffic  13194 non-null  object        
 7   Total_Sessions    13194 non-null  int64         
 8   Total_Bounces     13194 non-null  int64         
 9   Total_Duration    13194 non-null  float64       
dtypes: datetime64[ns](1), float64(1), int64(2), object(6)
memory usage: 1.0+ MB


In [20]:
# Create a CSV file as output.
GenStats.to_csv(r'general-stats_cleaned.csv', index=False)